## Combining dataframes from group members into one dataframe

In [226]:
import pandas as pd

In [227]:
jack = pd.read_csv('Jacks_df.csv', index_col = 'Unnamed: 0')

In [228]:
jack['county'] = jack['county_name'].apply(lambda x: x.lower() + ' county')
index = jack['county']

In [229]:
jack.drop(columns = 'county_name', inplace = True)

In [230]:
jack.set_index('county', inplace = True)

In [231]:
jack.shape

(254, 8)

In [232]:
voting_nums = pd.read_csv('./data/voting_numbers.csv', index_col = 'Unnamed: 0')

In [233]:
voting_nums.head()

,county,2012_reg_voters,2012_voted_num,2012_voted_perc,2012_early_vote_num,2012_early_vote_perc,2016_reg_voters,2016_voted_num,2016_voted_perc,2016_early_vote_num,2016_early_vote_perc,2020_reg_voters,2020_voted_num,2020_voted_perc,2020_early_vote_num,2020_early_vote_perc
156,anderson county,"26,494","16,168",61.03%,"9,322",35.19%,"28,111","16,887",60.07%,"10,792",38.39%,"29,274","19,155",65.43%,14459,75.48%
13,andrews county,"8,271","4,478",54.14%,"2,914",35.23%,"9,130","4,941",54.12%,"3,429",37.56%,"10,272",5856,57.01%,4776,81.56%
243,angelina county,"49,317","28,460",57.71%,"17,548",35.58%,"51,610","29,896",57.93%,"20,428",39.58%,"53,166",34628,65.13%,"28,676",82.81%
82,aransas county,"15,883","9,646",60.73%,"6,535",41.14%,"17,263","10,486",60.74%,"8,421",48.78%,"18,306","12,241",66.87%,"10,983",87.72%
67,archer county,"6,322","4,163",65.85%,"1,659",26.24%,"6,314","4,277",67.74%,"2,284",36.17%,"6,538","4,796",73.36%,"3,623",75.54%


In [234]:
voting_nums.drop_duplicates(inplace = True)

In [235]:
voting_nums.shape
voting_nums.set_index('county', inplace = True)

In [236]:
voting_nums.dtypes

2012_reg_voters         object
2012_voted_num          object
2012_voted_perc         object
2012_early_vote_num     object
2012_early_vote_perc    object
2016_reg_voters         object
2016_voted_num          object
2016_voted_perc         object
2016_early_vote_num     object
2016_early_vote_perc    object
2020_reg_voters         object
2020_voted_num          object
2020_voted_perc         object
2020_early_vote_num     object
2020_early_vote_perc    object
dtype: object

In [237]:
voting_nums.head()

,2012_reg_voters,2012_voted_num,2012_voted_perc,2012_early_vote_num,2012_early_vote_perc,2016_reg_voters,2016_voted_num,2016_voted_perc,2016_early_vote_num,2016_early_vote_perc,2020_reg_voters,2020_voted_num,2020_voted_perc,2020_early_vote_num,2020_early_vote_perc
county,,,,,,,,,,,,,,,
anderson county,"26,494","16,168",61.03%,"9,322",35.19%,"28,111","16,887",60.07%,"10,792",38.39%,"29,274","19,155",65.43%,14459,75.48%
andrews county,"8,271","4,478",54.14%,"2,914",35.23%,"9,130","4,941",54.12%,"3,429",37.56%,"10,272",5856,57.01%,4776,81.56%
angelina county,"49,317","28,460",57.71%,"17,548",35.58%,"51,610","29,896",57.93%,"20,428",39.58%,"53,166",34628,65.13%,"28,676",82.81%
aransas county,"15,883","9,646",60.73%,"6,535",41.14%,"17,263","10,486",60.74%,"8,421",48.78%,"18,306","12,241",66.87%,"10,983",87.72%
archer county,"6,322","4,163",65.85%,"1,659",26.24%,"6,314","4,277",67.74%,"2,284",36.17%,"6,538","4,796",73.36%,"3,623",75.54%


In [238]:
def percent_to_float(percent):
    """
    Converts string percentage to decimal proportion
    percent - string of number ending in "%" to convert to float
    """
    return float(percent[:-1]) / 100

In [239]:
perc_list = ['2012_voted_perc', '2012_early_vote_perc', 
             '2016_voted_perc', '2016_early_vote_perc', 
             '2020_voted_perc', '2020_early_vote_perc']

In [240]:
for col in perc_list:
    voting_nums[col] = voting_nums[col].apply(percent_to_float)

In [241]:
for col in voting_nums.columns:
    if voting_nums[col].dtype == 'object':
        voting_nums[col] = voting_nums[col].apply(lambda x: x.replace(",", ""))
    
for col in voting_nums.columns:
    voting_nums[col] = voting_nums[col].apply(pd.to_numeric)

In [242]:
jack.dtypes

2012_unemp                            float64
2016_unemp                            float64
2020_unemp                            float64
2012_var_state                        float64
2016_var_state                        float64
2020_var_state                        float64
median_income_2012_2016_16_dollars      int64
median_income_2020_16_dollars           int64
dtype: object

In [243]:
df = jack.merge(right = voting_nums, how = 'left', left_index = True, right_index = True)

In [244]:
df.head()

,2012_unemp,2016_unemp,2020_unemp,2012_var_state,2016_var_state,2020_var_state,median_income_2012_2016_16_dollars,median_income_2020_16_dollars,2012_reg_voters,2012_voted_num,...,2016_reg_voters,2016_voted_num,2016_voted_perc,2016_early_vote_num,2016_early_vote_perc,2020_reg_voters,2020_voted_num,2020_voted_perc,2020_early_vote_num,2020_early_vote_perc
county,,,,,,,,,,,,,,,,,,,,,
anderson county,0.036,0.043,0.056,-0.029,-0.005,-0.012,42146,44871,26494,16168,...,28111,16887,0.6007,10792.0,0.3839,29274,19155,0.6543,14459.0,0.7548
andrews county,0.031,0.047,0.082,-0.034,-0.001,0.014,70121,69369,8271,4478,...,9130,4941,0.5412,3429.0,0.3756,10272,5856,0.5701,4776.0,0.8156
angelina county,0.051,0.059,0.076,-0.014,0.011,0.008,44185,47917,49317,28460,...,51610,29896,0.5793,20428.0,0.3958,53166,34628,0.6513,28676.0,0.8281
aransas county,0.068,0.056,0.085,0.003,0.008,0.017,44851,49153,15883,9646,...,17263,10486,0.6074,8421.0,0.4878,18306,12241,0.6687,10983.0,0.8772
archer county,0.034,0.043,0.053,-0.031,-0.005,-0.015,62407,59010,6322,4163,...,6314,4277,0.6774,2284.0,0.3617,6538,4796,0.7336,3623.0,0.7554


In [245]:
df.shape

(254, 23)

In [246]:
voting_party_data = pd.read_csv('./data/voting_party_data.csv')

In [247]:
voting_party_data.shape

(254, 33)

In [248]:
voting_party_data.head()[:10]

,County,2020_rep_vote_count,2020_rep_vote_perc,2020_dem_vote_count,2020_dem_vote_perc,2020_lib_vote_count,2020_lib_vote_perc,2020_grn_vote_count,2020_grn_vote_perc,2020_oth_vote_count,...,2012_rep_vote_perc,2012_rep_vote_count,2012_dem_vote_perc,2012_dem_vote_count,2012_oth_vote_perc,2012_oth_vote_count,2012_vote_total,2012_CWPP,2016_CWPP,2020_CWPP
0,anderson county,"15,110",78.59%,"3,955",20.57%,134,0.70%,22,0.11%,6,...,75.64%,"12,262",23.52%,"3,813",0.85%,137,"16,212",0,0,0
1,andrews county,"4,943",84.31%,850,14.50%,60,1.02%,10,0.17%,0,...,81.19%,"3,639",17.74%,795,1.07%,48,"4,482",0,0,0
2,angelina county,"25,076",72.53%,"9,143",26.44%,274,0.79%,75,0.22%,6,...,71.47%,"20,303",27.58%,"7,834",0.95%,269,"28,406",0,0,0
3,aransas county,"9,239",75.17%,"2,916",23.73%,103,0.84%,31,0.25%,1,...,70.79%,"6,830",28.03%,"2,704",1.18%,114,"9,648",0,1,1
4,archer county,"4,300",89.66%,446,9.30%,45,0.94%,4,0.08%,1,...,86.46%,"3,600",12.61%,525,0.94%,39,"4,164",0,0,1


In [249]:
voting_party_data.set_index('County', inplace = True)

In [250]:
perc_list = [col for col in voting_party_data.columns if "perc" in col]

In [251]:
for col in perc_list:
    voting_party_data[col] = voting_party_data[col].apply(percent_to_float)

In [252]:
for col in voting_party_data.columns:
    if voting_party_data[col].dtype == 'object':
        voting_party_data[col] = voting_party_data[col].apply(lambda x: x.replace(",", ""))
    
for col in voting_party_data.columns:
    voting_party_data[col] = voting_party_data[col].apply(pd.to_numeric)

In [253]:
voting_party_data.dtypes

2020_rep_vote_count       int64
2020_rep_vote_perc      float64
2020_dem_vote_count       int64
2020_dem_vote_perc      float64
2020_lib_vote_count       int64
2020_lib_vote_perc      float64
2020_grn_vote_count       int64
2020_grn_vote_perc      float64
2020_oth_vote_count       int64
2020_oth_vote_perc      float64
2020_votes_total          int64
2016_rep_vote_counte      int64
2016_rep_vote_perc      float64
2016_dem_vote_count       int64
2016_dem_vote_perc      float64
2016_lib_vote_count       int64
2016_lib_vote_perc      float64
2016_grn_vote_count       int64
2016_grn_vote_perc      float64
2016_oth_vote_count       int64
2016_oth_vote_perc      float64
2016_votes_total          int64
2012_rep_vote_perc      float64
2012_rep_vote_count       int64
2012_dem_vote_perc      float64
2012_dem_vote_count       int64
2012_oth_vote_perc      float64
2012_oth_vote_count       int64
2012_vote_total           int64
2012_CWPP                 int64
2016_CWPP                 int64
2020_CWP

In [254]:
voting_party_data.index.rename('county', inplace=True)

In [255]:
df = df.merge(right = voting_party_data, how = 'left', left_index = True, right_index = True)


In [256]:
df.shape

(254, 55)

In [257]:
df.head()

,2012_unemp,2016_unemp,2020_unemp,2012_var_state,2016_var_state,2020_var_state,median_income_2012_2016_16_dollars,median_income_2020_16_dollars,2012_reg_voters,2012_voted_num,...,2012_rep_vote_perc,2012_rep_vote_count,2012_dem_vote_perc,2012_dem_vote_count,2012_oth_vote_perc,2012_oth_vote_count,2012_vote_total,2012_CWPP,2016_CWPP,2020_CWPP
county,,,,,,,,,,,,,,,,,,,,,
anderson county,0.036,0.043,0.056,-0.029,-0.005,-0.012,42146,44871,26494,16168,...,0.7564,12262,0.2352,3813,0.0085,137,16212,0,0,0
andrews county,0.031,0.047,0.082,-0.034,-0.001,0.014,70121,69369,8271,4478,...,0.8119,3639,0.1774,795,0.0107,48,4482,0,0,0
angelina county,0.051,0.059,0.076,-0.014,0.011,0.008,44185,47917,49317,28460,...,0.7147,20303,0.2758,7834,0.0095,269,28406,0,0,0
aransas county,0.068,0.056,0.085,0.003,0.008,0.017,44851,49153,15883,9646,...,0.7079,6830,0.2803,2704,0.0118,114,9648,0,1,1
archer county,0.034,0.043,0.053,-0.031,-0.005,-0.015,62407,59010,6322,4163,...,0.8646,3600,0.1261,525,0.0094,39,4164,0,0,1


In [258]:
asr = pd.read_csv('./data/asr_12_16_20.csv')

In [259]:
asr.tail()

,county,total_2012_18 to 27,total_2012_28 to 37,total_2012_38 to 47,total_2012_48 to 57,total_2012_58 to 67,total_2012_68 to 77,total_2012_78 to 87,total_2012_88 to 97,total_2016_18 to 27,...,hispanic_female_2020_58 to 67,hispanic_female_2020_68 to 77,hispanic_female_2020_78 to 87,hispanic_female_2020_88 to 97,total_male_2012,total_male_2016,total_male_2020,total_female_2012,total_female_2016,total_female_2020
249,wood county,0.1043,0.0874,0.1050,0.1384,0.1566,0.1355,0.0725,0,0.1080,...,0.0040,0.0014,0.0011,0.0003,0.4922,0.4927,0.5009,0.5078,0.5073,0.4991
250,yoakum county,0.1293,0.1250,0.1170,0.1330,0.0928,0.0568,0.0384,0,0.1342,...,0.0236,0.0145,0.0077,0.0022,0.4961,0.5056,0.4966,0.5039,0.4944,0.5034
251,young county,0.1124,0.1050,0.1157,0.1481,0.1250,0.0848,0.0706,0,0.1119,...,0.0087,0.0030,0.0014,0.0007,0.4943,0.5009,0.4920,0.5057,0.4991,0.5080
252,zapata county,0.1578,0.1277,0.1114,0.0986,0.0816,0.0546,0.0325,0,0.1551,...,0.0419,0.0339,0.0124,0.0035,0.5008,0.5064,0.4912,0.4992,0.4936,0.5088
253,zavala county,0.1590,0.1152,0.1096,0.1105,0.0993,0.0587,0.0442,0,0.1522,...,0.0416,0.0352,0.0151,0.0050,0.4924,0.4957,0.5101,0.5076,0.5043,0.4899


In [260]:
asr.set_index('county', inplace = True)

In [261]:
df = df.merge(right = asr, how = 'left', left_index = True, right_index = True)

In [262]:
df.head()

,2012_unemp,2016_unemp,2020_unemp,2012_var_state,2016_var_state,2020_var_state,median_income_2012_2016_16_dollars,median_income_2020_16_dollars,2012_reg_voters,2012_voted_num,...,hispanic_female_2020_58 to 67,hispanic_female_2020_68 to 77,hispanic_female_2020_78 to 87,hispanic_female_2020_88 to 97,total_male_2012,total_male_2016,total_male_2020,total_female_2012,total_female_2016,total_female_2020
county,,,,,,,,,,,,,,,,,,,,,
anderson county,0.036,0.043,0.056,-0.029,-0.005,-0.012,42146,44871,26494,16168,...,0.0036,0.0026,0.0007,0.0003,0.6062,0.6078,0.6198,0.3938,0.3922,0.3802
andrews county,0.031,0.047,0.082,-0.034,-0.001,0.014,70121,69369,8271,4478,...,0.0163,0.0096,0.0041,0.0013,0.4992,0.5017,0.5158,0.5008,0.4983,0.4842
angelina county,0.051,0.059,0.076,-0.014,0.011,0.008,44185,47917,49317,28460,...,0.0074,0.0038,0.0014,0.0003,0.4911,0.4950,0.4908,0.5089,0.5050,0.5092
aransas county,0.068,0.056,0.085,0.003,0.008,0.017,44851,49153,15883,9646,...,0.0131,0.0088,0.0044,0.0011,0.4946,0.4930,0.4995,0.5054,0.5070,0.5005
archer county,0.034,0.043,0.053,-0.031,-0.005,-0.015,62407,59010,6322,4163,...,0.0030,0.0008,0.0014,0.0006,0.5032,0.4956,0.4936,0.4968,0.5044,0.5064


In [263]:
df.tail()

,2012_unemp,2016_unemp,2020_unemp,2012_var_state,2016_var_state,2020_var_state,median_income_2012_2016_16_dollars,median_income_2020_16_dollars,2012_reg_voters,2012_voted_num,...,hispanic_female_2020_58 to 67,hispanic_female_2020_68 to 77,hispanic_female_2020_78 to 87,hispanic_female_2020_88 to 97,total_male_2012,total_male_2016,total_male_2020,total_female_2012,total_female_2016,total_female_2020
county,,,,,,,,,,,,,,,,,,,,,
wood county,0.047,0.053,0.066,-0.018,0.005,-0.002,47814,52727,27438,17543,...,0.0040,0.0014,0.0011,0.0003,0.4922,0.4927,0.5009,0.5078,0.5073,0.4991
yoakum county,0.035,0.048,0.119,-0.030,0.000,0.051,56655,57996,3919,2128,...,0.0236,0.0145,0.0077,0.0022,0.4961,0.5056,0.4966,0.5039,0.4944,0.5034
young county,0.038,0.044,0.053,-0.027,-0.004,-0.015,46978,48744,11535,7316,...,0.0087,0.0030,0.0014,0.0007,0.4943,0.5009,0.4920,0.5057,0.4991,0.5080
zapata county,0.073,0.108,0.124,0.008,0.060,0.056,36976,33397,7435,3529,...,0.0419,0.0339,0.0124,0.0035,0.5008,0.5064,0.4912,0.4992,0.4936,0.5088
zavala county,0.111,0.139,0.141,0.046,0.091,0.073,26639,30128,8566,3653,...,0.0416,0.0352,0.0151,0.0050,0.4924,0.4957,0.5101,0.5076,0.5043,0.4899


In [264]:
df.shape

(254, 421)

In [265]:
df.to_csv('combined_df.csv')

In [269]:
df.isnull().sum()

2012_unemp           0
2016_unemp           0
2020_unemp           0
2012_var_state       0
2016_var_state       0
                    ..
total_male_2016      0
total_male_2020      0
total_female_2012    0
total_female_2016    0
total_female_2020    0
Length: 421, dtype: int64